In [1]:
import pandas as pd

from mod.logs import Logger
from mod.dao import MyConn, SqlStatement
from data_lake import PT_TABLE, OP_TABLE, DX_TABLE


In [3]:
# Prepare modules
l = Logger()
conn = MyConn("127.0.0.1", "hecon", l, True)
builder = SqlStatement()

In [4]:
# Load data from database
pt = conn.wrap(builder.read_data, table=PT_TABLE)
op = conn.wrap(builder.read_data, table=OP_TABLE)
dx = conn.wrap(builder.read_data, table=DX_TABLE)


        select * from hecon.pt_dl
        
wrapper result: (('total', '2021-12-01', 5), ('total', '2021-12-02', 11), ('total', '2021-12-03', 11), ('total', '2021-12-04', 11), ('total', '2021-12-05', 18), ('total', '2021-12-06', 26), ('total', '2021-12-07', 30), ('total', '2021-12-08', 33), ('total', '2021-12-09', 36), ('total', '2021-12-10', 32), ('total', '2021-12-11', 31), ('total', '2021-12-12', 32), ('total', '2021-12-13', 34), ('total', '2021-12-14', 38), ('total', '2021-12-15', 36), ('total', '2021-12-16', 36), ('total', '2021-12-17', 33), ('total', '2021-12-18', 29), ('total', '2021-12-19', 30), ('total', '2021-12-20', 34), ('total', '2021-12-21', 38), ('total', '2021-12-22', 38), ('total', '2021-12-23', 40), ('total', '2021-12-24', 32), ('total', '2021-12-25', 24), ('total', '2021-12-26', 33), ('total', '2021-12-27', 37), ('total', '2021-12-28', 42), ('total', '2021-12-29', 41), ('total', '2021-12-30', 40), ('total', '2021-12-31', 34), ('total', '2022-01-01', 28), ('total', '2

In [16]:
# Exclude 
# 1. Zero inflation
script = """
    SELECT 
      id,
      SUM(value = 0) AS count_of_zeros,
      COUNT(*) as total_count,
      (SUM(value = 0) / COUNT(*)) AS zero_ratio
    FROM
      dx_dl
    GROUP BY
      id;
    HAVING
      zero_ratio < 0.9
"""